In [1]:
from gurobipy import *

In [2]:
##### Preparing Data 
# Demand (8 products)
# setting period = 24+1 (initial status t=0)
Demands = [[0, 52, 53, 52, 53, 55, 55, 55, 55, 55, 55, 55, 55, 20, 20, 20, 20, 0, 0, 0, 0, 0, 0, 0, 0], 
           [0, 8, 9, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
           [0, 87, 88, 87, 88, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85, 85], 
           [0, 291, 291, 291, 292, 291, 291, 291, 292, 291, 291, 291, 292, 291, 291, 291, 292, 291, 291, 291, 292, 291, 291, 291, 292], 
           [0, 75, 75, 75, 75, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80], 
           [0, 25, 25, 25, 25, 26, 26, 26, 27, 28, 29, 29, 29, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55],
           [0, 222, 223, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225],
           [0, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25]]

# initiazlize inventory level for t=0  (arbitruarly decided. about 1/4 of average demand)
BegInv = [50, 10, 90, 200, 70, 25, 150, 25]

SetupCost = 500 
InvHoldCost = 1  
holders = 13 

molds = range(len(Demands))
periods = range(len(Demands[0]))

In [3]:
##### Create Model
m = Model("Scheduling")

In [4]:
##### Add Decision Variables
# number of molds of type i in conveyer during period t
usage = []
for i in molds:
    usage.append([])
    for t in periods:
        usage[i].append(m.addVar(lb=0, vtype = GRB.INTEGER, name = "Mold%d.%d" % (i,t)))

In [5]:
# Linearnize absoulte value in obj. funcion - Yit(+), Yit(-)
change = []
for i in molds:
    change.append([])
    for t in periods:
        change[i].append([])
        for y in range(2):
            change[i][t].append(m.addVar(lb=0, vtype = GRB.INTEGER, obj=(0.5*SetupCost), name = "Change%d.%d.%d" % (i,t,y)))

In [6]:
# Ending inventory level of products from mold type i at perid t 
EndInv = []
for i in molds:
    EndInv.append([])
    for t in periods:
        EndInv[i].append(m.addVar(lb=0, vtype = GRB.INTEGER, obj=InvHoldCost, name = "Inv%d.%d" % (i,t)))

In [7]:
# The objective is to minimize the total setup cost and inventory holding cost
m.modelSense = GRB.MINIMIZE

In [8]:
# Update model to integrate new variables
m.update()

In [9]:
# initiazlize inventory level for t=0
for i in molds:
    EndInv[i][0] = BegInv[i]
    
# initializ Yit(+), Yit(-) when t=0 
for i in molds:
    for y in range(2):
        change[i][0][y]=0

In [10]:
##### Define Constraints
# number of holders
for t in periods[1:]:
    m.addConstr(quicksum(usage[i][t] for i in molds) == holders)

In [11]:
# inventory level and demand equation 
for i in molds:
    for t in periods[1:]:
        m.addConstr((EndInv[i][t-1] + 60*usage[i][t] - Demands[i][t]) == EndInv[i][t])

In [12]:
# usage & linearnized variables (Y) constraint
for i in molds:
    for t in periods[1:]:
        m.addConstr(usage[i][t]-usage[i][t-1] == (change[i][t][0] - change[i][t][1]))

In [13]:
# maximum number of molds of each type i available (upper bound for number of molds)
mold_constraint = {0:12, 1:5, 2:8, 3:15, 4:4, 5:7, 6:14, 7:3}
for t in periods[1:]:
    for mold, vals in mold_constraint.items():
        expr = LinExpr(usage[mold][t]) 
        m.addConstr(expr, GRB.LESS_EQUAL, vals) 

In [14]:
##### solve 
m.optimize()

Optimize a model with 600 rows, 800 columns and 1720 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 6e+01]
  Objective range [1e+00, 2e+02]
  Bounds range    [0e+00, 0e+00]
  RHS range       [2e+00, 3e+02]
Presolve removed 262 rows and 302 columns
Presolve time: 0.00s
Presolved: 338 rows, 498 columns, 1149 nonzeros
Variable types: 0 continuous, 498 integer (0 binary)

Root relaxation: objective 1.411691e+04, 397 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 14116.9091    0  256          - 14116.9091      -     -    0s
     0     0 15495.8745    0  249          - 15495.8745      -     -    0s
     0     0 15554.3108    0  256          - 15554.3108      -     -    0s
     0     0 16385.2312    0  232          - 16385.2312      -     -    0s
     0     0 16394.3423    0  231          - 16394.3423      -     -    0s
     0     0 16436.6

In [15]:
# vars = m.getVars()
# vars
# m.getObjective()
# for v in m.getVars():
#     print(v.x)

In [16]:
##### Print solutions
#
print "Setup cost per mold: %g" % SetupCost
print "Inventory Holding cost per item: %g" % InvHoldCost

# Total Cost
print "Total Costs is: %g" % m.objVal

# Setup
result_change = []
for i in molds:
    for t in periods[1:]:
        # one mold out Y(+) and another mold in Y(-), but only one mold change occurs. Thus, loop Y only in range(1) 
        for y in range(1):
            if change[i][t][y].x >= 1.0:
                result_change.append(t)
print "Before these periods a mold change is required:"
print sorted(result_change)
                
# Ending Inventory
for i in molds:
    End = EndInv[i][-1].x
    print "Mold %g - ending inventory after the last period: %g" %(i,End)

Setup cost per mold: 500
Inventory Holding cost per item: 1
Total Costs is: 20101
Before these periods a mold change is required:
[2, 4, 4, 5, 8, 9, 11, 11, 13, 13, 14, 18, 21]
Mold 0 - ending inventory after the last period: 40
Mold 1 - ending inventory after the last period: 35
Mold 2 - ending inventory after the last period: 20
Mold 3 - ending inventory after the last period: 110
Mold 4 - ending inventory after the last period: 30
Mold 5 - ending inventory after the last period: 65
Mold 6 - ending inventory after the last period: 215
Mold 7 - ending inventory after the last period: 145
